In [1]:
import pandas as pd
import numpy as np

print("🔗 FUSION DES DONNÉES IQVIA CLEANED")
print("=" * 45)

🔗 FUSION DES DONNÉES IQVIA CLEANED


In [3]:
# Cell 1: Chargement des données cleaned
df_campagne = pd.read_csv("../../data/cleaned/iqvia_campagne_cleaned.csv")
df_couverture = pd.read_csv("../../data/cleaned/iqvia_couverture_cleaned.csv")
df_doses = pd.read_csv("../../data/cleaned/iqvia_doses_cleaned.csv")

# Conversion des dates
df_campagne['date'] = pd.to_datetime(df_campagne['date'])
df_doses['date'] = pd.to_datetime(df_doses['date'])

print(f"📊 Données cleaned chargées :")
print(f"   Campagne : {df_campagne.shape}")
print(f"   Couverture : {df_couverture.shape}")
print(f"   Doses : {df_doses.shape}")


📊 Données cleaned chargées :
   Campagne : (15, 5)
   Couverture : (156, 5)
   Doses : (3096, 6)


In [4]:
# Cell 2: Préparation pour fusion
print("\n🔄 Préparation pour fusion...")

# 1. Pivot des données de campagne
df_campagne_pivot = df_campagne.pivot_table(
    index=['campagne', 'date'], 
    columns='variable', 
    values='valeur', 
    aggfunc='first'
).reset_index()

# 2. Pivot des données de couverture
df_couverture_pivot = df_couverture.pivot_table(
    index=['region', 'code_region', 'groupe_age'], 
    columns='variable', 
    values='valeur', 
    aggfunc='first'
).reset_index()

# 3. Pivot des données doses
df_doses_pivot = df_doses.pivot_table(
    index=['campagne', 'date', 'jour', 'groupe_age'], 
    columns='variable', 
    values='valeur', 
    aggfunc='first'
).reset_index()

print("✅ Données pivotées")


🔄 Préparation pour fusion...
✅ Données pivotées


In [5]:
# Cell 3: Création du dataset fusionné
print("\n🎯 Création du dataset fusionné...")

# Dataset principal : doses agrégées par date et groupe d'âge
df_doses_agg = df_doses_pivot.groupby(['date', 'groupe_age', 'campagne']).agg({
    'DOSES(J07E1)': 'sum',
    'ACTE(VGP)': 'sum'
}).reset_index()

# Ajout des composants temporels
df_doses_agg['annee'] = df_doses_agg['date'].dt.year
df_doses_agg['mois'] = df_doses_agg['date'].dt.month
df_doses_agg['jour_semaine'] = df_doses_agg['date'].dt.dayofweek
df_doses_agg['semaine_annee'] = df_doses_agg['date'].dt.isocalendar().week

print(f"✅ Dataset principal : {df_doses_agg.shape}")


🎯 Création du dataset fusionné...
✅ Dataset principal : (1010, 9)


In [6]:
# Cell 4: Ajout des données de campagne
print("\n📊 Ajout des données de campagne...")

# Agrégation des données de campagne par date
campagne_daily = df_campagne_pivot.groupby('date').agg({
    'DOSES(J07E1)': 'mean',
    'ACTE(VGP)': 'mean',
    'UNIVERS': 'mean',
    'PHARMACIES': 'mean',
    'POURCENTAGE': 'mean'
}).reset_index()

# Fusion avec les données principales
df_iqvia_fusionne = pd.merge(
    df_doses_agg,
    campagne_daily,
    on='date',
    how='left',
    suffixes=('', '_campagne')
)

print(f"✅ Dataset fusionné : {df_iqvia_fusionne.shape}")



📊 Ajout des données de campagne...
✅ Dataset fusionné : (1010, 14)


In [7]:
# Cell 5: Sauvegarde
print("\n💾 Sauvegarde...")

df_iqvia_fusionne.to_csv("../../data/cleaned/iqvia_fusionne_cleaned.csv", index=False)

print("✅ Dataset fusionné sauvegardé : iqvia_fusionne_cleaned.csv")

# Cell 6: Validation finale
print("\n📋 VALIDATION FINALE")
print("=" * 30)

print(f"📊 Shape final : {df_iqvia_fusionne.shape}")
print(f"📅 Période : {df_iqvia_fusionne['date'].min()} à {df_iqvia_fusionne['date'].max()}")
print(f"👥 Groupes d'âge : {df_iqvia_fusionne['groupe_age'].nunique()}")
print(f"🗓️ Campagnes : {df_iqvia_fusionne['campagne'].nunique()}")

print("\n🔍 Aperçu du dataset fusionné :")
display(df_iqvia_fusionne.head())

print("\n🎉 FUSION TERMINÉE !")


💾 Sauvegarde...
✅ Dataset fusionné sauvegardé : iqvia_fusionne_cleaned.csv

📋 VALIDATION FINALE
📊 Shape final : (1010, 14)
📅 Période : 2021-10-22 00:00:00 à 2025-01-28 00:00:00
👥 Groupes d'âge : 2
🗓️ Campagnes : 4

🔍 Aperçu du dataset fusionné :


variable,date,groupe_age,campagne,DOSES(J07E1),ACTE(VGP),annee,mois,jour_semaine,semaine_annee,DOSES(J07E1)_campagne,ACTE(VGP)_campagne,UNIVERS,PHARMACIES,POURCENTAGE
0,2021-10-22,65 ans et plus,2021-2022,567525,117217,2021,10,4,42,NaN,NaN,NaN,NaN,NaN
1,2021-10-22,moins de 65 ans,2021-2022,111876,24537,2021,10,4,42,NaN,NaN,NaN,NaN,NaN
2,2021-10-23,65 ans et plus,2021-2022,234990,42728,2021,10,5,42,NaN,NaN,NaN,NaN,NaN
3,2021-10-23,moins de 65 ans,2021-2022,54861,12205,2021,10,5,42,NaN,NaN,NaN,NaN,NaN
4,2021-10-24,65 ans et plus,2021-2022,2005,351,2021,10,6,42,NaN,NaN,NaN,NaN,NaN



🎉 FUSION TERMINÉE !
